**This notebook contains the following sections:**

**#01** Importing libraries and data

**#02** Plotting choropleth

**#03** Reflection

#01 Importing libraries and data

In [213]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import folium
import json

In [214]:
#Importing data
path = '/Users/gingermoore/Documents/06-2025 Airbnb Dallas'

In [215]:
#Import GeoJSON file
with open('/Users/gingermoore/Documents/06-2025 Airbnb Dallas/02 Data/Original Data/neighbourhoods.geojson') as f:
    dallas_geo = json.load(f)

In [216]:
#Import listings data
df = pd.read_pickle(os.path.join(path, '02 Data', 'Transformed Data', 'CleanDallasDataFinal.pkl'))

In [217]:
pd.set_option('display.max_columns', None)

In [218]:
df.head()

listing_id                           name  \
0      61878  MODERN LIVING AND FURNISHINGS   
1      61878  MODERN LIVING AND FURNISHINGS   
2      61878  MODERN LIVING AND FURNISHINGS   
3      61878  MODERN LIVING AND FURNISHINGS   
4      61878  MODERN LIVING AND FURNISHINGS   

                                         description  \
0  Close to downtown and Uptown.  Fast and conven...   
1  Close to downtown and Uptown.  Fast and conven...   
2  Close to downtown and Uptown.  Fast and conven...   
3  Close to downtown and Uptown.  Fast and conven...   
4  Close to downtown and Uptown.  Fast and conven...   

                               neighborhood_overview host_id host_since  \
0  Enjoy the heart of Dallas right out your door ...  300211 2010-11-26   
1  Enjoy the heart of Dallas right out your door ...  300211 2010-11-26   
2  Enjoy the heart of Dallas right out your door ...  300211 2010-11-26   
3  Enjoy the heart of Dallas right out your door ...  300211 2010-11-26   
4  Enjoy the heart of Dallas right out your door ...  300211 2010-11-26   

  host_location  host_response_time  host_response_rate  host_acceptance_rate  \
0   Roanoke, TX  within a few hours               100.0                  88.0   
1   Roanoke, TX  within a few hours               100.0                  88.0   
2   Roanoke, TX  within a few hours               100.0                  88.0   
3   Roanoke, TX  within a few hours               100.0                  88.0   
4   Roanoke, TX  within a few hours               100.0                  88.0   

  host_neighbourhood    district  latitude  longitude property_type  \
0           Oak Lawn  District 2   32.8169  -96.82018  Entire condo   
1           Oak Lawn  District 2   32.8169  -96.82018  Entire condo   
2           Oak Lawn  District 2   32.8169  -96.82018  Entire condo   
3           Oak Lawn  District 2   32.8169  -96.82018  Entire condo   
4           Oak Lawn  District 2   32.8169  -96.82018  Entire condo   

         room_type  accommodates  bathrooms  bedrooms  beds  min_nights  \
0  Entire home/apt             3        1.0       1.0   2.0          30   
1  Entire home/apt             3        1.0       1.0   2.0          30   
2  Entire home/apt             3        1.0       1.0   2.0          30   
3  Entire home/apt             3        1.0       1.0   2.0          30   
4  Entire home/apt             3        1.0       1.0   2.0          30   

   max_nights  number_of_reviews  rating_score  accuracy_score  clean_score  \
0        1125                 56          4.71            4.79          4.6   
1        1125                 56          4.71            4.79          4.6   
2        1125                 56          4.71            4.79          4.6   
3        1125                 56          4.71            4.79          4.6   
4        1125                 56          4.71            4.79          4.6   

   checkin_score  comm_score  location_score  value_score  host_listings  \
0           4.85        4.93            4.75         4.78              1   
1           4.85        4.93            4.75         4.78              1   
2           4.85        4.93            4.75         4.78              1   
3           4.85        4.93            4.75         4.78              1   
4           4.85        4.93            4.75         4.78              1   

   host_entire_homes  host_private_rooms  host_shared_rooms  \
0                  1                   0                  0   
1                  1                   0                  0   
2                  1                   0                  0   
3                  1                   0                  0   
4                  1                   0                  0   

   reviews_per_month superhost_bool  instant_book_bool       date  price  \
0               0.32           True              False 2025-04-17   75.0   
1               0.32           True              False 2025-04-18   75.0   
2               0.32       

**Note:** I've already done all the data cleaning and consistency checks, this data is reliable. I do need to create a column that shows the number of listings per district.

In [220]:
df_map = df.groupby('district').agg({'listing_id': 'nunique'}).reset_index()
df_map.columns = ['district', 'listing_count']

In [221]:
df_map

district  listing_count
0    District 1            517
1   District 10            171
2   District 11            309
3   District 12            152
4   District 13            236
5   District 14           1082
6    District 2           1686
7    District 3            101
8    District 4            166
9    District 5             42
10   District 6            307
11   District 7            239
12   District 8             83
13   District 9            177

**#02 Plotting choropleth**

In [250]:
# Setup a folium map, zoomed in on Dallas
map = folium.Map(location = [32.7767, -96.7970], zoom_start = 11)

folium.Choropleth(
    geo_data = dallas_geo, 
    data = df_map,
    columns = ['district', 'listing_count'],
    key_on = 'feature.properties.neighbourhood',
    fill_color = 'Blues', fill_opacity=0.8, line_opacity=0.1,
    legend_name = "Number_of_Listings").add_to(map).add_to(map)

In [252]:
#Adding labels to the districts when you hover over them (thanks ChatGPT)!

# Create a lookup dictionary
count_lookup = dict(zip(df_map['district'], df_map['listing_count']))

# Add listing_count to GeoJSON properties
for feature in dallas_geo['features']:
    district = feature['properties']['neighbourhood']
    feature['properties']['listing_count'] = count_lookup.get(district, 0)  # fallback to 0 if missing

folium.GeoJson(
    dallas_geo,
    name='Districts',
    tooltip=folium.GeoJsonTooltip(fields=['neighbourhood', 'listing_count'], aliases=['District:', 'Listings:'])
).add_to(map)

In [254]:
map

In [256]:
map.save('plot_data.html')

**#03 Reflection**

This map answers one of my research questions: which districts have the most listings? Answer: Districts 1, 2, and 14. I want to know if the price in these districts is higher! I may look into that soon.